In [1]:
!pip install matplotlib tensorflow_model_optimization tf_keras image_classifiers
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
import tensorflow_model_optimization as tfmot
from tqdm import tqdm
import numpy as np
import tensorflow as tf
import tf_keras as keras
from tf_keras.applications import ResNet50V2
from tf_keras.datasets import cifar100
from tf_keras import Sequential, Input
from tf_keras.layers import Dense, Dropout, RandomFlip, RandomTranslation, RandomRotation,RandomBrightness, RandomContrast, RandomZoom, GlobalAveragePooling2D
from tf_keras.applications.resnet_v2 import preprocess_input
from tf_keras.models import Model
from classification_models.models_factory import ModelsFactory
import tempfile

2024-04-21 18:22:20.034910: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-21 18:22:20.100689: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-21 18:22:21.033483: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
class KerasModelsFactory(ModelsFactory):
    @staticmethod
    def get_kwargs():
        return {
            'backend': keras.backend,
            'layers': keras.layers,
            'models': keras.models,
            'utils': keras.utils,
        }
# Pour faire marcher image-classifiers avec tf_keras, la version compatibilité de tf.keras, nécéssaire pour tflite
Classifiers = KerasModelsFactory()

In [3]:
n_epoch = 10
batch_size = 100
num_classes = 100
n_images = 50000 # Pour l'entrainement, et 10000 pour le test

In [4]:
!mc cp s3/afeldmann/projet_cnam/modele_enseignant.keras /home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras
model_enseignant = Sequential([
    Input((224,224,3)),
    ResNet50V2(include_top=False, weights='imagenet', pooling="avg"),
    Dropout(0.25),
    Dense(256, activation="sigmoid", kernel_regularizer = keras.regularizers.L1(0.001)),
    Dropout(0.5),
    Dense(num_classes, activation="softmax", kernel_regularizer = keras.regularizers.L2(0.001))
])
# Keras 3.1.1 est buggé et le chargement direct ne marche pas ici, même si les poids sont bien enregistrés
model_enseignant.load_weights("/home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras")

model_enseignant.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

...nant.keras: 135.73 MiB / 135.73 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 102.12 MiB/s 1s

2024-04-21 18:22:29.050392: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13290 MB memory:  -> device: 0, name: NVIDIA A2, pci bus id: 0000:b1:00.0, compute capability: 8.6


In [5]:
def ResNet18():
    resnet18, preprocess_input = Classifiers.get('resnet18')
    resnet = resnet18((224, 224, 3), weights='imagenet', include_top=False)
    resnet_output = GlobalAveragePooling2D()(resnet.output)
    resnet = Model(inputs=resnet.input, outputs=resnet_output)
    return resnet

def new_modele_resnet():
    model = Sequential([
        Input((224,224,3)),
        ResNet18(),
        Dropout(0.25),
        Dense(256, activation="sigmoid", kernel_regularizer = keras.regularizers.L1(0.001)),
        Dropout(0.5),
        Dense(num_classes, activation="softmax", kernel_regularizer = keras.regularizers.L2(0.001))
    ])
    model.compile(metrics=['accuracy'])
    return model

In [6]:
!mc cp s3/afeldmann/projet_cnam/model_etudiant_t1_a0.keras /home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant.keras
model_etudiant = new_modele_resnet()
model_etudiant.load_weights("/home/onyxia/work/projet_distillation_cnam/sauvegardes/model_etudiant.keras")
model_etudiant.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

...1_a0.keras: 43.54 MiB / 43.54 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 96.12 MiB/s 0s

In [7]:
def preprocessing(image, label):
    image = tf.image.resize(image, (224, 224))
    label = tf.squeeze(tf.one_hot(label, depth = num_classes), axis = 0)
    return  image, label

augmentation_donnees_keras = Sequential([
    RandomFlip("horizontal"),
    RandomTranslation(0.2,0.2),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomContrast(0.2),
    RandomBrightness(0.2,value_range=(0,1))
])

def augmentation_donnees(image, label):
    return augmentation_donnees_keras(image/255.0, training = True)*255.0, label

def preprocess_resnet(image, label):
    return preprocess_input(image), label

train_dataset, test_dataset = cifar100.load_data()
train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset).map(preprocessing).cache().shuffle(n_images).batch(batch_size).map(augmentation_donnees, num_parallel_calls = tf.data.AUTOTUNE).map(preprocess_resnet, num_parallel_calls = tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices(test_dataset).map(preprocessing).batch(batch_size).map(preprocess_resnet, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

In [8]:
end_step = np.ceil(n_images / batch_size).astype(np.int32) * n_epoch

pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.20,
                                                               final_sparsity=0.60,
                                                               begin_step=0,
                                                               end_step=end_step)
}

model_etudiant_pruning = tfmot.sparsity.keras.prune_low_magnitude(model_etudiant, **pruning_params)

In [9]:
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep()
]

model_etudiant_pruning.compile(optimizer=keras.optimizers.SGD(learning_rate=0.005), loss='categorical_crossentropy', metrics=['accuracy'])

model_etudiant_pruning.fit(train_dataset, callbacks=callbacks, epochs=n_epoch)

Epoch 1/10


2024-04-21 18:23:46.181087: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900
I0000 00:00:1713723845.273642   62107 service.cc:145] XLA service 0x7f4d51cb6f60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1713723845.273786   62107 service.cc:153]   StreamExecutor device (0): NVIDIA A2, Compute Capability 8.6
I0000 00:00:1713723845.414995   62107 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


500/500 [==============================] - 328s 507ms/step - loss: 9.2837 - accuracy: 0.6845
Epoch 2/10
500/500 [==============================] - 266s 528ms/step - loss: 8.3904 - accuracy: 0.7168
Epoch 3/10
500/500 [==============================] - 254s 505ms/step - loss: 7.7240 - accuracy: 0.7263
Epoch 4/10
500/500 [==============================] - 251s 499ms/step - loss: 7.1917 - accuracy: 0.7276
Epoch 5/10
500/500 [==============================] - 268s 533ms/step - loss: 6.7698 - accuracy: 0.7250
Epoch 6/10
500/500 [==============================] - 267s 530ms/step - loss: 6.4331 - accuracy: 0.7233
Epoch 7/10
500/500 [==============================] - 266s 528ms/step - loss: 6.1692 - accuracy: 0.7172
Epoch 8/10
500/500 [==============================] - 265s 526ms/step - loss: 5.9525 - accuracy: 0.7187
Epoch 9/10
500/500 [==============================] - 262s 520ms/step - loss: 5.7695 - accuracy: 0.7194
Epoch 10/10
500/500 [==============================] - 266s 529ms/step - lo

In [22]:
model_etudiant_pruning.evaluate(test_dataset)

100/100 [==============================] - 17s 120ms/step - loss: 5.2153 - accuracy: 0.7783


[5.215271949768066, 0.7782999873161316]

In [23]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_etudiant_pruning)
_, pruned_keras_file = tempfile.mkstemp('.h5')
keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)

/tmp/ipykernel_61794/1381957240.py:3: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)


In [24]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
pruned_tflite_model = converter.convert()
pruned_tflite_file = "/home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_etudiant.tflite"
with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)


INFO:tensorflow:Assets written to: /tmp/tmp11ff4qp6/assets


INFO:tensorflow:Assets written to: /tmp/tmp11ff4qp6/assets


Saved pruned TFLite model to: /home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_etudiant.tflite


W0000 00:00:1713727158.022523   61794 tf_tfl_flatbuffer_helpers.cc:390] Ignored output_format.
W0000 00:00:1713727158.022550   61794 tf_tfl_flatbuffer_helpers.cc:393] Ignored drop_control_dependency.
2024-04-21 19:19:18.023046: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp11ff4qp6
2024-04-21 19:19:18.032863: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2024-04-21 19:19:18.032880: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmp11ff4qp6
2024-04-21 19:19:18.100194: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-04-21 19:19:18.107817: I tensorflow/cc/saved_model/loader.cc:234] Restoring SavedModel bundle.
2024-04-21 19:19:18.305235: I tensorflow/cc/saved_model/loader.cc:218] Running initialization op on SavedModel bundle at path: /tmp/tmp11ff4qp6
2024-04-21 19:19:18.363115: I tensorflow/cc/saved_model/loader.cc

In [25]:
test_batches = test_dataset.unbatch().batch(1)

interpreter = tf.lite.Interpreter(model_path=pruned_tflite_file)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

predictions = []

test_labels, test_imgs = [], []
for img, label in tqdm(test_batches.take(1000)):
    interpreter.set_tensor(input_index, img)
    interpreter.invoke()
    predictions.append(interpreter.get_tensor(output_index))

    test_labels.append(label.numpy()[0])
    test_imgs.append(img)


score = 0
for item in range(0,len(predictions)):
  prediction=np.argmax(predictions[item])
  label = np.argmax(test_labels[item])
  if prediction==label:
    score=score+1

print("Out of 1000 predictions I got " + str(score) + " correct")

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
999it [00:54, 20.39it/s]2024-04-21 19:20:26.822163: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
1000it [00:54, 18.47it/s]

Out of 1000 predictions I got 784 correct
